In [1]:
import numpy as np
import pandas as pd
import os
import re
import spacy
import csv
import pickle
import csv

pd.set_option('display.width', 120)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', 90)

In [2]:
import sys  

default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)  
sys.stdout = default_stdout
sys.stderr = default_stderr
sys.setdefaultencoding('utf8')

In [3]:
nlp = spacy.load('en')

In [4]:
def recursiveSplit(line, sentences):
    if (len(line) <= 10):
        return []
    elif (len(sentences) < 2) or (len(line) < 250):
        return [line]
    else:
        allLengths = [len(s) for s in sentences]
        leftSide = 0
        rightSide = sum(allLengths)
        diffs = []
        for l in allLengths:
            leftSide = leftSide + l
            rightSide = rightSide - l
            diffs.append(abs(leftSide - rightSide))
        splitIndex = diffs.index(min(diffs))

        previous = ' '.join([str(s) for s in sentences[:splitIndex+1]])
        previousSent = sentences[:splitIndex+1]
        upcoming = ' '.join([str(s) for s in sentences[splitIndex+1:]])
        upcomingSent = sentences[splitIndex+1:]

        splitSentence = []
        splitSentence += recursiveSplit(previous, previousSent)
        splitSentence += recursiveSplit(upcoming, upcomingSent)   
        return splitSentence

In [49]:
files = ["0516_carrier", "0616_jersey", "0716_convention", "0816_immigration", "0916_economy",
          "0916_security", "0916_victory", "1016_accusation", "1016_smith", "0217_dump"]

allText = []
for filename in files:
    toRead = "speeches/"+filename+".txt"
    fileText = []
    with open (toRead, "r") as myfile:
        for line in myfile:
            text = nlp(unicode(line))
            sentences = [s for s in text.sents]
            fileText += recursiveSplit(line, sentences)
    print filename, ": ", len(fileText)

    allText += fileText
print "FINAL LENGTH: ", len(allText)

with open('speeches.csv', 'wb+') as f:
    myWriter = csv.writer(f)
    for q, a in zip(allText, allText[1:]):
        myWriter.writerow([a, q, 'speeches'])

0516_carrier :  111
0616_jersey :  64
0716_convention :  196
0816_immigration :  254
0916_economy :  132
0916_security :  105
0916_victory :  50
1016_accusation :  163
1016_smith :  61
0217_dump :  6074
FINAL LENGTH:  7210


In [53]:
with open("twitter.csv","r") as f:
    reader = csv.reader(f,delimiter = ",")
    rows = list(reader)
    print len(rows)

19405


In [ ]:
files = ["0516_carrier", "0616_jersey", "0716_convention", "0816_immigration", "0916_economy",
         "0916_security", "0916_victory", "1016_accusation", "1016_smith"]
all_dfs = []

for file in files:
    filename = "dirty/speeches/"+file+".txt"
    data = []
    with open (filename, "r") as myfile:
        for line in myfile:
            data.append(line)
    df = pd.DataFrame({"answer": data})    
    df['answer'].replace(to_replace="\n", value="", inplace=True, regex=True)
    df['query'] = np.where(df['answer'].str.contains("Mr. Trump, your"), df['answer'].shift(2), df['answer'].shift(1))
    df['source'] = file
    df = df.dropna()
    all_dfs.append(df)
    print file + " " + str(df.shape)

final_df = pd.concat(all_dfs)
final_df.to_csv("clean/speeches.csv", index=False, header=True)

In [ ]:
# from spacy.en import English
# nlp = English()

# full = full.dropna(axis=0)
# full = full[full['Content'].str.len() < 600]
preX = full['Content']
preX = preX.apply(lambda row: row.strip())
preX = preX.apply(lambda row: row.replace('-', ' '))
parsed = preX.apply(lambda row: nlp(unicode(row, 'utf-8', 'ignore')) )
reindexed = parsed.reset_index(drop=True)
# X = reindexed
postX = []
for idx, sentence in enumerate(preX):
    found_entities = []
    for ent in reindexed[idx].ents:
        if ent.label_ == 'ORG':
            found_entities.append( (ent.text.encode('utf-8'), 'ORG') )
        if ent.label_ == 'PERSON':
            found_entities.append( (ent.text.encode('utf-8'), 'PERSON') )
        if ent.label_ == 'MONEY':
            found_entities.append( (ent.text.encode('utf-8'), 'MONEY') )
    for word in found_entities:
        sentence = sentence.replace(word[0], word[1])
    postX.append(sentence)
    if (idx%2500 == 0):
        print idx

postX = pd.Series(postX)
parsed = postX.map(lambda row: nlp(unicode(row, 'utf-8', 'ignore')) )
tokenized = parsed.map(lambda row: [token.orth_ for token in row])
X = tokenized
print len(X)